# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1YvrGoaMvEsuGSKdGWZ8t8jJNEGZfj3bx?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [33]:
# !pip install selenium / Utilizado este comando para instalar selenium
# Foi baixado o chromedriver e inserido na pasta do python

from selenium import webdriver  # cria o navegador / Aqui foi importado apenas o webdriver da biblioteca do selenium
from selenium.webdriver.common.by import By  # localizar elementos (itens de um site)
from selenium.webdriver.common.keys import Keys  # permite utilizar teclas do teclado

navegador = webdriver.Chrome()

# Passo 1 - Entrar no Google
navegador.get("https://www.google.com.br/")  # entra no site do google no navegador criado pelo webdriver

# Passo 2 - Pesquisar a cotação do dólar
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")  # procura o campo de pesquisa e escreve a pesquisa
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)  # pressiona enter para pesquisar

cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')  # procura o dado de valor da cotação e armaneza na variável
print(cotacao_dolar)

# Passo 3 - Pegar a cotação do Euro
navegador.get("https://www.google.com.br/")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# Passo 4 - Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

5.4146
5.546905751000001
297.63


In [34]:
# Passo 5 - Atualizar a base de dados com as novas cotações
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")

display(tabela)

Produtos  Preço Original  Moeda  Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar        5          4999.95    1.40   
1   Carro Renault         4500.00   Euro        6         27000.00    2.00   
2   Notebook Dell          899.99  Dólar        5          4499.95    1.70   
3          IPhone          799.00  Dólar        5          3995.00    1.70   
4      Carro Fiat         3000.00   Euro        6         18000.00    1.90   
5  Celular Xiaomi          480.48  Dólar        5          2402.40    2.00   
6        Joia 20g           20.00   Ouro      350          7000.00    1.15   

   Preço de Venda  
0        6999.930  
1       54000.000  
2        7649.915  
3        6791.500  
4       34200.000  
5        4804.800  
6        8050.000

In [35]:
# Passo 5.1 - atualizar a cotação de acordo com a moeda correspondente
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# Passo 5.2 - atualizar o preço de compra (Preço Original * Cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Passo 5.3 - atualizar o preço de venda (Preço de Compra * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

Produtos  Preço Original  Moeda     Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar    5.414600      5414.545854    1.40   
1   Carro Renault         4500.00   Euro    5.546906     24961.075880    2.00   
2   Notebook Dell          899.99  Dólar    5.414600      4873.085854    1.70   
3          IPhone          799.00  Dólar    5.414600      4326.265400    1.70   
4      Carro Fiat         3000.00   Euro    5.546906     16640.717253    1.90   
5  Celular Xiaomi          480.48  Dólar    5.414600      2601.607008    2.00   
6        Joia 20g           20.00   Ouro  297.630000      5952.600000    1.15   

   Preço de Venda  
0     7580.364196  
1    49922.151759  
2     8284.245952  
3     7354.651180  
4    31617.362781  
5     5203.214016  
6     6845.490000

In [36]:
# Passo 6 - Salvar criando um novo arquivo com os valores atualizados
tabela.to_excel("Produtos Novos.xlsx", index=False)
navegador.quit()  # finaliza o navegador criado